# RUNNING AND PLOTTING A QUERY FOR MULTIPLE DATES

In [2]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq

## Set these:

In [ ]:
project_id = ""
sql = ""

In [3]:
# Connecting to BigQuery
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████| 10/10 [00:00<00:00, 41.37rows/s]


In [4]:
def build_query(query, date_as_string):
    return query.format(date = date_as_string)

In [5]:
def add_leading_zero(month_or_day):
    if len(month_or_day) == 1:
        month_or_day = '0' + month_or_day
    return month_or_day

In [6]:
# only designed for valid dates
def add_one_to_date(date_as_string):
    # TODO: deal with leap years (year%4=0 && year%100!=0 or something)
    new_date_as_string = ''
    days_in_month = {
        '01':31,
        '02':28,
        '03':31,
        '04':30,
        '05':31,
        '06':30,
        '07':31,
        '08':31,
        '09':30,
        '10':31,
        '11':30,
        '12':31
    }
    year = date_as_string[0:4]
#     print("Year: " + year)
    month = date_as_string[4:6]
#     print("Month: " + month)
    day = date_as_string[6:8]
#     print("Day: " + day)
    # if it's not the last day in the month
    if int(day) < days_in_month[month]:
        day = add_leading_zero(str(int(day) + 1))
        new_date_as_string = year + month + day
    # if it's the last day in the month and it's not the last month
    elif (int(day) == days_in_month[month]) and (month != '12'):
        month = add_leading_zero(str(int(month) + 1))
        day = '01'
        new_date_as_string = year + month + day
    # if it's the last day of the month and it's the last month   
    elif (int(day) == days_in_month[month]) and (month == '12'):
        year = add_leading_zero(str(int(year) + 1))
        month = '01'
        day = '01'
        new_date_as_string = year + month + day
    else:
        print("???")
    return new_date_as_string

In [8]:
def create_date_list(start_date, end_date):
#     example start date: '20210114'; end date could be current date
    date_list = []
    if start_date == end_date:
        date_list.append(start_date)
    while start_date != end_date:
        date_list.append(start_date)
        start_date = add_one_to_date(start_date)
    return date_list

In [10]:
def iterate_through_dates(start_date, end_date, query):
    date_list = create_date_list(start_date, end_date)
    query_list = []
    for date in date_list:
        query_list.append(build_query(query, date))
    return query_list

In [16]:
def plot_bar_graph(query, x_axis, y_axis, color):
    df = pandas_gbq.read_gbq(query, project_id=project_id)
    df.plot.bar(x=x_axis, y=y_axis, color=color)

In [17]:
def query_to_bar_graphs(query, start_date, end_date, x_axis, y_axis, color):
    query_list = iterate_through_dates(start_date, end_date, query)
    for query in query_list:
        plot_bar_graph(query, x_axis, y_axis, color)

In [30]:
def query_to_tables(query, start_date, end_date):
    query_list = iterate_through_dates(start_date, end_date, query)
    for query in query_list:
        print(pandas_gbq.read_gbq(query, project_id=project_id))

In [48]:
# make table of element_count over time for a given event
def query_to_element_count_over_time(query, start_date, end_date, index):
    query_list = iterate_through_dates(start_date, end_date, query)
#     row_list = []
    element_count_array = []
    for query in query_list:
        df = pandas_gbq.read_gbq(query, project_id=project_id)
        print(df)
        element_count_array.append(df.iloc[index]['element_count'])
    print(element_count_array)
    return element_count_array
        
#         row = df.iloc[index, :]
#         row_list.append(row)
#     print(row_list)
#     return row_list

In [38]:
def plot_line_graph(query, x_axis, y_axis, color):
    df = pandas_gbq.read_gbq(query, project_id=project_id)
    df.plot.line(x=x_axis, y=y_axis, color=color)

In [64]:
def query_to_element_count_graph_over_time(query, start_date, end_date, index, color):
    element_count_list = query_to_element_count_over_time(query, start_date, end_date, index)
#     checking that the plotting is correct
#     element_count_list = [145015098, 0, 126753792, 0, 110191447, 0, 92114747]
    
#     todo: find event_name and/or element by index
#     x_axis = "event_name"
#     y_axis = element_count_list
#     plot_line_graph(query, x_axis, y_axis, color)

#     todo: fix dates
#     x = np.arange(int(start_date), int(end_date))
    
#     todo: fix
#     x = np.arange(1, 8)
    x = np.array(create_date_list(start_date, end_date))

    y = np.array(element_count_list)
    plt.title("element_count over time")
    plt.xlabel("date")
    plt.ylabel("element_count")
    plt.plot(x, y, color=color)
    plt.show()